<a href="https://colab.research.google.com/github/Chao-coder/Dahua/blob/main/%E9%81%97%E7%95%99%E8%A1%A8%E5%A4%84%E7%90%860526.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries we are going to use
import numpy as np # linear algebra
import pandas as pd # data processing
from plotly.offline import iplot, init_notebook_mode #visualization
import plotly.express as px #visualization
from plotly.subplots import make_subplots #visualization
import plotly.graph_objects as go #visualization
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization
from sklearn import preprocessing #label encoder 
from sklearn.model_selection import train_test_split #split data
from sklearn.linear_model import LogisticRegression #model
from sklearn.model_selection import cross_val_score #cross validation
from sklearn.metrics import classification_report #reports 
from sklearn import metrics #confusion metrics
from sklearn.model_selection import GridSearchCV #get best parameters


In [2]:
#import the filenames
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
# convert file into dataset
# data = pd.read_csv('/content/wcc.csv')
data = pd.read_excel('/content/0526遗留明细-DY.xlsx')
# data = pd.read_csv('/content/0519.csv')

In [4]:
#explore first five rows in the dataset
data.head()

,PI号,制单人,创建日期,排产日期,订单号,一生二标识,一生二订单号,辖区,组别,业绩归属国,...,预计船期,预计出库日期,行总金额USD,发货计划,付款条件,返工标识,原始需求日期,排产意见,产品系列,产品主型号
0,C2762772303101D9963A,胡春思,2023-03-13,2023-03-13,2023031300139,N,NaN,俄语区,运营管理组,亚美尼亚,...,2023/05/27 16:00:00,2023/05/24 00:00:00,6096,凑柜发货,OA 90,NaN,2023-03-25 00:00:00,NaN,VTH5 Series,VTH5221D-S2
1,C2762772303101D9963A,胡春思,2023-04-25,2023-04-25,2023042500191,N,NaN,俄语区,分销业务部,亚美尼亚,...,2023/05/27 16:00:00,2023/05/24 00:00:00,482,凑柜发货,OA 90,NaN,2023-05-20 00:00:00,NaN,KT Series,KTA02
2,C2762772303101D9963A,胡春思,2023-04-03,2023-04-03,2023040300019,N,NaN,俄语区,分销业务部,亚美尼亚,...,2023/05/27 16:00:00,2023/05/24 00:00:00,40200,凑柜发货,OA 90,NaN,2023-04-15 00:00:00,NaN,SD49 Series,SD49225ZB
3,C2762772303101D9963A,胡春思,2023-03-31,2023-03-31,2023033100028,N,NaN,俄语区,分销业务部,亚美尼亚,...,2023/05/27 16:00:00,2023/05/24 00:00:00,3864,凑柜发货,OA 90,NaN,2023-04-15 00:00:00,NaN,VTO3 Series,VTO3211D-S2
4,C2762772303101D9963A,胡春思,2023-03-31,2023-03-31,2023033100028,N,NaN,俄语区,分销业务部,亚美尼亚,...,2023/05/27 16:00:00,2023/05/24 00:00:00,6544,凑柜发货,OA 90,NaN,2023-04-15 00:00:00,NaN,Hardened PoE Switch,PFS3106-4ET-60-V2


In [5]:
#information about dataset, type, columns names, null 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389 entries, 0 to 1388
Columns: 109 entries, PI号 to 产品主型号
dtypes: datetime64[ns](3), float64(29), int64(10), object(67)
memory usage: 1.2+ MB


In [6]:
df = data[['实际交易客户','业务员','客户PO号','物料编码','内部型号','规格型号','订单行数量','单价','行合计','币种','状态','统筹交期','彩盒Box Weight (kg), ','彩盒Box Volume(m³)','每箱台数Pcs/Carton','毛件体（长度）','毛件体（宽度）','毛件体（高度）','毛件体（体积）',]]

In [7]:
业务员 = df['业务员'].unique()
业务员

array(['程思', '黄伟杰', '赵安然', 'Tsvetkov Pavel', '孙世林', '汪喆', '李嘉慧',
       'Kachalin Maxim', '纪洋', '杨宇', '王昱周'], dtype=object)

In [8]:
df.head()

,实际交易客户,业务员,客户PO号,物料编码,内部型号,规格型号,订单行数量,单价,行合计,币种,状态,统筹交期,"彩盒Box Weight (kg),",彩盒Box Volume(m³),每箱台数Pcs/Carton,毛件体（长度）,毛件体（宽度）,毛件体（高度）,毛件体（体积）
0,Mercury,程思,ML202301-02,1.0.01.15.11062-0024,DHI-VTH5221DW-S2,DHI-VTH5221DW-S2,80,76.2,6096.0,USD,入库,2023-05-15,0.78,0.005868,10,0.774,0.324,0.234,0.058682
1,Mercury,程思,ML202301-02,1.0.01.15.11509-0061,DHI-KTA02,DHI-KTA02,10,48.2,482.0,USD,入库,2023-05-23,1.03,0.006739,10,0.585,0.320,0.360,0.067392
2,Mercury,程思,ML202301-02,1.0.01.07.13754-0001,DH-SD49225-HNR-ZB,DH-SD49225XA-HNR-S2,200,201.0,40200.0,USD,入库,2023-04-30,4.84,0.031438,1,0.267,0.267,0.441,0.031438
3,Mercury,程思,ML202301-02,1.0.01.15.11384-0016,DHI-VTO3211D-P1-S2,DHI-VTO3211D-P1,60,64.4,3864.0,USD,入库,2023-05-06,0.74,0.003739,10,0.424,0.323,0.273,0.037388
4,Mercury,程思,ML202301-02,1.0.01.20.10614-0019,DH-PFS3106-4ET-60-V2,DH-PFS3106-4ET-60,160,40.9,6544.0,USD,入库,2023-04-30,1.15,0.005747,8,0.549,0.253,0.331,0.045975


In [ ]:
# 通过.loc 筛选出某一列中的其中多项子项
# 王昱周 = df.loc[df['业务员'].isin(['王昱周','杨宇'])]
# 王昱周.head()

,实际交易客户,业务员,客户PO号,物料编码,内部型号,规格型号,订单行数量,单价,行合计,币种,状态,统筹交期,"彩盒Box Weight (kg),",彩盒Box Volume(m³),每箱台数Pcs/Carton,毛件体（长度）,毛件体（宽度）,毛件体（高度）,毛件体（体积）
767,Jupiter,杨宇,DH-TR2023001,1.0.01.04.41025,DH-IPC-HFW5442HP-ZHE-2712F-DC12AC24V-S3,DH-IPC-HFW5442H-ZHE,92,160.0,14720.0,USD,入库,2023-05-15,3.010,0.019379,6,0.635,0.451,0.406,0.116272
768,Jupiter,杨宇,DH-TR2023001,1.0.01.34.13055-9002,DHI-TPC-SDA8641-B50Z45-BM-S22,DHI-TPC-SDA8641-B50Z45-BM-S22,5,4500.0,22500.0,USD,入库,2023-05-20,21.020,0.156043,1,0.580,0.380,0.708,0.156043
769,Jupiter,杨宇,DH-TR2023001,1.0.99.12.10062,DHI-LM24-F200-C4,DHI-LM24-F200,25,75.0,1875.0,USD,入库,2023-04-15,4.143,0.039000,1,0.602,0.116,0.490,0.034218
770,Jupiter,杨宇,DH-TR2023001,1.0.01.20.11055,DH-IS4210-8GT-120,DH-IS4210-8GT-120,27,175.0,4725.0,USD,入库,2023-04-23,1.410,0.005656,20,0.530,0.396,0.539,0.113125
771,Jupiter,杨宇,DH-TR2023001,1.0.01.20.10728,DH-PFS4218-16ET-240-V3,DH-PFS4218-16ET-240,1,120.0,120.0,USD,入库,2023-04-20,4.700,0.028599,5,0.584,0.549,0.446,0.142995


In [9]:
程思 = df[df['业务员']=='程思']
黄伟杰 = df[df['业务员']=='黄伟杰']
赵安然 = df[df['业务员']=='赵安然']
TP = df[df['业务员']=='Tsvetkov Pavel']
孙世林 = df[df['业务员']=='孙世林']
汪喆 = df[df['业务员']=='汪喆']
李嘉慧 = df[df['业务员']=='李嘉慧']
KM = df[df['业务员']=='Kachalin Maxim']
纪洋 = df[df['业务员']=='纪洋']
# 杨宇 = df[df['业务员']=='杨宇']
王昱周 = df.loc[df['业务员'].isin(['王昱周','杨宇'])]
# 王昱周 = df[df['业务员']=='王昱周']
李杨 = df.loc[df['业务员'].isin(['李杨','Tsvetkov Pavel','Kachalin Maxim'])]

# 李杨 = df[df['业务员']=='李杨']


In [11]:
李杨.head()

,实际交易客户,业务员,客户PO号,物料编码,内部型号,规格型号,订单行数量,单价,行合计,币种,状态,统筹交期,"彩盒Box Weight (kg),",彩盒Box Volume(m³),每箱台数Pcs/Carton,毛件体（长度）,毛件体（宽度）,毛件体（高度）,毛件体（体积）
382,Cassiopeia,Tsvetkov Pavel,YY202301-01,1.0.01.04.40733#0001,IPC-HFW5541EP-ZE-0735-S3,BMFK535,16,140.0,2240.0,USD,入库,2023-05-23,1.70,0.009672,4,0.390,0.320,0.310,0.038688
383,Cassiopeia,Tsvetkov Pavel,YY202301-02,1.1.03.81.20814,PFL0550-E6D,PFL0550-E6D,200,45.0,9000.0,USD,入库,2050-01-01,0.28,0.001140,1,0.127,0.088,0.080,0.000894
384,Cassiopeia,Tsvetkov Pavel,YY202301-02,1.0.01.04.41723-0001,IPC-HFW5241EP-ZE-0735-S3-Synergo,B2/35,5000,115.0,575000.0,USD,在制,2023-06-15,0.00,0.013841,10,0.910,0.389,0.391,0.138410
385,Cassiopeia,Tsvetkov Pavel,YY202301-02,1.1.03.81.20949,PFA113,Syn-KP420,50,11.0,550.0,USD,入库,2050-01-01,1.14,0.002170,1,0.063,0.061,0.440,0.001691
386,Cassiopeia,Tsvetkov Pavel,YY202301-02,1.0.01.04.41719-0001,IPC-HDBW5241EP-ZE-27135-DC12AC24V-S3-Synergo,D2/12,3485,96.0,334560.0,USD,在制,2023-06-15,0.00,0.012768,4,0.485,0.390,0.270,0.051070


In [ ]:
import time
date = time.strftime('%Y_%m_%d',time.localtime())
print(date)
type(date)

2023_05_25


str

In [ ]:
# 存储路径及文件名

filename = "date.xlsx"
directory = "/content/"
file_savepath = directory + filename + date
print(file_savepath)

/content/date.xlsx2023_05_25


In [12]:
# 黄伟杰.to_csv('hwj.csv')/content/0524遗留明细-DY.xlsx
程思.to_excel('/content/程思0526.xlsx')
黄伟杰.to_excel('/content/黄伟杰0526.xlsx')
赵安然.to_excel('/content/赵安然0526.xlsx')
# TP.to_excel('/content/Tsvetkov Pavel_0524.xlsx')
孙世林.to_excel('/content/孙世林0526.xlsx')
汪喆.to_excel('/content/汪喆0526.xlsx')
李嘉慧.to_excel('/content/李嘉慧0526.xlsx')
# KM.to_excel('/content/Kachalin Maxim_0524.xlsx')
纪洋.to_excel('/content/纪洋0526.xlsx')
# 杨宇.to_excel('/content/0524遗留表/杨宇0524.xlsx')
王昱周.to_excel('/content/王昱周0526.xlsx')
李杨.to_excel('/content/李杨0526.xlsx')
